italicized text# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2020 - TP1 - Méthodes de recherche 
### Membres de l'équipe
    - Didier Blach-Laflèche #xxx
    - Maxim Pecherskiy #xxx
    - Marc-Antoine Huet #xxx


## Directives de remise
Le travail sera réalisé en équipe de trois. Vous remettrez un fichier nommé TP1\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boite de remise de travaux sur moodle. 

Le fichier devra être remis avant le **29 septembre à 23h55**. Tout travail en retard sera pénalisé
de 10\% de points en moins par jour de retard.





## Snail Labyrinth

*Snail Labyrinth* est un jeu dont le but est de guider des escargots dans un labyrinthe jusqu'à leur sortie personnelle. La taille des labyrinthes varie. Chaque escargot peut se déplacer dans cinq directions: vers le haut, vers le bas, vers la droite, vers la gauche et rester sur place. 

A chaque étape, chaque escargot à la possibilité d'effectuer un mouvement.
Les escargots ne peuvent pas se déplacer dans une case où se déplace un autre escargot. De plus, les escargots laissent derrière ils une substance toxique et ne peuvent donc pas se déplacer dans une case qui a déjà été occupée par un autre escargot.

A titre d'illustation, les images ci-dessous donnent un exemple d'une configuration initiale (Figure 1) et d'une configuration où deux mouvements ont eté realisés (Figure 2). Les étapes sont les suivantes:

- Etape 1: le escargot A se déplace vers la droite et le escargot B vers la droite.
- Etape 2: le escargot A se déplace vers la droite et le escargot B vers la droite.

<table><tr>
<td> <img src="https://imgur.com/3W4iDSk.png" alt="Drawing"/> <figcaption>Figure 1 - Example d'une configuration initiale.</figcaption></td>
<td> <img src="https://imgur.com/3uaD3AO.png" alt="Drawing"/><figcaption>Figure 2 - Example de deux mouvements.</figcaption> </td>
</tr></table>

L'objectif de ce TP est d'écrire un programme qui permet aux escargots d'atteindre leur destination en un **minimum de mouvements**.

Pour ce faire, il vous est demandé de compléter les deux classes fournies en Python. Vous pouvez réimplémenter ces classes-là et faire le TP dans un autre langage si vous le désirez.

## 1. Représentation du problème (10pts)

Le problème est défini par les règles suivantes. Les colonnes sont numérotées de gauche à droite de 0 à $m$ et les lignes sont numérotées de haut en bas, de 0 à $n$. Un escargot est considérée *résolue* lorsqu'il arrive à sa propre sortie.

L'état du labyrinthe est représenté par un objet de la classe **State** qui contient les variables d'instance suivantes: 

- `pos`: vecteur indiquant la position de chaque escargot sur la grille;
- `venoms`: ensemble des positions du poison laissé par les escargots;
- `d`: liste indiquant la direction du déplacement de chaque escargot fait à partir de l'état précédent pour atteindre l'état courant (0: rester en place, 1: vers le haut, 2: vers la droite, 3: vers le bas et 4: vers la gauche);
- `prev`: l'état précédent



In [1]:
import numpy as np
import math
import copy

class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance(self, other):
        return math.fabs(other.x - self.x) + math.fabs(other.y - self.y)

    def distance_y(self, other):
        return math.fabs(other.y - self.y)

    def distance_x(self, other):
        return math.fabs(other.x - self.x)
        
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __repr__(self):
        return "(%d,%d)" % (self.x, self.y)
    
    def __str__(self):
        return "(%d,%d)" % (self.x, self.y)
    
    def __hash__(self):
        return int(self.x*1+self.y*2)

class State:
    
    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position du escargot i;
        """
        self.pos = np.array(copy.deepcopy(pos))
        """
        venoms garde les positions du poisons
        """
        self.venoms = set()
        
        """
        d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.d = self.prev = None
        self.nb_moves = 0
        self.h = 0

    """
    Constructeur d'un nouvel état à partir d'un mouvement (d)
    """    
    def move(self, d):
        # DONE
        movedState = State(self.pos)
        movedState.prev = self
        movedState.venoms = copy.copy(self.venoms)
        movedState.d = d
        movedState.nb_moves = self.nb_moves

        for i in range(len(d)):
            if d[i] != 0:
                movedState.venoms.add(self.pos[i])
            if d[i] == 1:
                movedState.pos[i].x -= 1
            elif d[i] == 2:
                movedState.pos[i].y += 1
            elif d[i] == 3:
                movedState.pos[i].x += 1
            elif d[i] == 4:
                movedState.pos[i].y -= 1

        movedState.nb_moves += 1
        return movedState
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de limaces")

        return np.array_equal(self.pos, other.pos) and (len(self.venoms) == len(other.venoms)  and self.venoms == other.venoms)
        
    def __hash__(self):
        h = 0
        for snail_position in self.pos:
            h += snail_position.x*25 + snail_position.y*30
        for venom_position in self.venoms:
            h += snail_position.x*25 + snail_position.y*30
        return int(h)
    
    def __lt__(self, other):
        return (self.nb_moves + self.h) < (other.nb_moves + other.h)

    def __repr__(self):
        out = ""
        for index, limace in enumerate(self.pos):
            out += "Escargot %d (%d,%d) | " % (index+1, limace.x, limace.y)
        return "State: " + out[:-3]
    
    def __str__(self):
        out = ""
        for index, limace in enumerate(self.pos):
            out += "Escargot %d (%d,%d) | " % (index+1, limace.x, limace.y)
        return out[:-3]

#### Implémentation 
   - Complétez la fonction ***move*** en déplaçant les escargots de 1 case dans la bonne direction (d[i]= 0, 1, 2, 3 ou 4). Nous supposons que le déplacement est possible lorsque la fonction est appelée. **Attention** : Cette fonction doit créer un nouvel objet *State* et définir les variables d'instance `prev`, `venoms` et `d`.
   
Vous pouvez tester votre code avec la fonction ***test1()*** dans la cellule suivante. Assurez-vous d'avoir déjà roulé la cellule avec la définition de la classe **State** avant. Vous devriez obtenir les résultats suivants:
```
True
(3,0)  (3,1)
True
True
```

In [2]:
def test1():
    b = True
    s0 = State([Position(0,0), Position(3,0)])
    s1 = s0.move([2,2])
    print(s1.prev == s0)
    b = b and s1.prev == s0
    print(s0.pos[1], " ", s1.pos[1])
    s1 = s1.move([0,0])
    print(s1 == s1.prev)
    b = b and s1 == s1.prev
    s1 = s1.move([3,3]).move([4,4]).move([1,1])
    s2 = s0.move([3,3]).move([2,2]).move([1,1]).move([0,0]).move([4,4])
    print(s1 == s2)
    b = b and s1 == s2
    
    print("\nrésultat correct" if b else "\nmauvais résultat")
    
test1()

True
(3,0)   (3,1)
True
True

résultat correct


## 2. Mouvements possibles (25pts)
Pour représenter le problème, nous allons utiliser la classe **Labyrinth** qui contient les champs suivants:

- `nb_lines`: le nombre de lignes de la grille;
- `nb_columns`: le nombre de colonnes de la grille;
- `exits`: un vecteur contenant la position de la sortie de chaque escargot (escargot $i$ sort à la sortie `exits[i]`);
- `walls`: un vecteur contenant la position de chaque murs dan la grille;
- `free_pos`: une matrice $n\times m$ contenant une valeur booléenne permettant de savoir si la case est libre ou non.

Toutes les informations pour un escargot se trouve au même indice $i$ dans chacun des vecteurs.

In [3]:
from collections import deque
import heapq
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt
import itertools

class Labyrinth:
    
    def __init__(self, lines, columns ,exits, walls):
        self.nb_lines = lines
        self.nb_columns = columns
        self.exits = exits
        self.walls = walls
        
        self.free_pos = None

    """
    Détermine si tous les escargots ont atteint leur position de sortie 
    """
    def success(self, state):
        # DONE
        return self.exits == [*state.pos]

    """
    Met à jour les positions libres dans le labyrinthe pour l'état actuel
    """
    def init_positions(self, state):
        self.free_pos = np.ones((self.nb_lines, self.nb_columns), dtype=bool)
        # DONE
        for wall in self.walls:
            self.free_pos[wall.x,wall.y] = False
        for venom in state.venoms:
            self.free_pos[venom.x,venom.y] = False 
        for pos in state.pos:
            self.free_pos[pos.x,pos.y] = False     
            
        return self.free_pos    
            
    """
    Détermine les déplacements possibles pour un escargot
    """
    def possible_moves_snail(self, snail_position, exit_position, other_exits):
        moves = []
        # DONE
        newPositions = []
        if snail_position == exit_position :
             moves = [(0, exit_position)]

        else: 
            posUpdates = [(0,0), (-1,0), (0,1), (1,0), (0,-1)] #initialise tous les déplacements posssibles

            for posUpdate in posUpdates:
                #Créations des nouvelles positions possibles
                newPositions.append(Position(snail_position.x + posUpdate[0], snail_position.y + posUpdate[1]))

            for i in range(1, len(newPositions)):
                #Si la nouvelle position est dans les limites du labyrinthe
                if (newPositions[i].x >= 0 and newPositions[i].x < self.nb_lines) and (newPositions[i].y >= 0 and newPositions[i].y < self.nb_columns) :
                    #Si la nouvelle position n'est pas la sortie d'un autre escargot, ni un mur
                    if (bool(newPositions[i] not in other_exits) and self.free_pos[newPositions[i].x, newPositions[i].y] and newPositions[i]):
                        moves.append((i, newPositions[i]))

            #Ajout du movement trivial   
            moves.append((0, newPositions[0]))   

        return moves


    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        # DONE

        ## Test si une sortie est bloquée
        # Positions aux alentours des sorties (si à l'extérieur du labyl. -> retiré)
        surroundingPositions = []
        occupied_counter = []
        for i, exit in enumerate(self.exits):
            occupied_counter.append(0)
            liste = [Position(exit.x + 1, exit.y), 
                    Position(exit.x - 1, exit.y),
                    Position(exit.x, exit.y + 1),
                    Position(exit.x, exit.y - 1)]
            for position in liste:  # Si la position est à l'extérieur du labyl.
              if position.x >= self.nb_lines or position.x < 0 or position.y >= self.nb_columns or position.y < 0:
                occupied_counter[i] += 1
                liste.remove(position)
            surroundingPositions.append(liste)
                  
        for i, exitSurroundingPositions in enumerate(surroundingPositions):
          if state.pos[i] != self.exits[i]:  # Si l'escargot n'est pas à sa sortie
            otherSnailsPositions = list(state.pos)
            otherSnailsPositions.pop(i)
            for surroundingPosition in exitSurroundingPositions: 
              if surroundingPosition in self.exits or surroundingPosition in self.walls or surroundingPosition in state.venoms or surroundingPosition in otherSnailsPositions:
                  occupied_counter[i] += 1
            if occupied_counter[i] == 4:
              return []
        
        ## Extraction de TOUS les mouvements possibles de chaque escargot
        moves = []  # Liste [[Escargot 1], [Escargot 2], ...] 
                    # [Escargot i] = [(mouv 1), (mouv 2), ...]
        for snailIndex in range(len(state.pos)) :
            snail_position = state.pos[snailIndex]
            exit_position = self.exits[snailIndex]
            other_exits = copy.copy(self.exits)
            other_exits.pop(snailIndex)
            snailMoves = self.possible_moves_snail(snail_position, exit_position, other_exits)
            # Si un escargot est bloqué et pas à sa sortie
            if snail_position != exit_position and (len(snailMoves) == 1) and (snailMoves[0][0] == 0):
                return []
            else:
                moves.append(snailMoves)
        
        ## Création d'une liste avec TOUTES les combinaisons des mouvements des différents escargots
        queue = [[]]
        for snailIndex in range(len(moves)):   # Pour chaque escargot
            tempQueue = []
            for i in range(len(queue)):  # Pour chaque liste dans la queue
                mouvementCombination = queue.pop()   # on pop la derniere liste de combinaison de mouvement
                for j in range(len(moves[snailIndex])):   # Pour chaque mouvement de cet escargot
                    #On concatene la combinaison des mouvement des escargot précédent
                    # avec un mouvement de cet escargot ci.
                    newMouvComb = mouvementCombination + [list(moves[snailIndex][j])] 
                    tempQueue.append(newMouvComb)

            # la queue temporé se cette itération devient la queue de la prochaine itération
            queue = copy.copy(tempQueue)   

        ## Rejet des combinaisons illégales de mouvements    
        removeIndex = []
        for combIndex in range(len(queue)): #Pour chaque combinaison

            # Test si tous les escargots restent immobiles
            noMovementCounter = 0
            for snailIndex in range(len(queue[combIndex])):
                if (queue[combIndex][snailIndex][0] == 0):
                    noMovementCounter += 1
            if (noMovementCounter == len(queue[combIndex])):
                removeIndex.append(combIndex)
            else:
              newPositionsList = []
              for snailIndex in range(len(queue[combIndex])):
                  newPositionsList.append(queue[combIndex][snailIndex][1])

              for snailIndex in range(len(queue[combIndex])): #Pour chaque escargot dans la combinaison.
                  liste = copy.copy(newPositionsList)
                  snailPosition = liste.pop(snailIndex)
                  otherSnailsPositions = list(copy.copy(state.pos))
                  otherSnailsPositions.pop(snailIndex)
                  
                  #Flag d'un mouvement de 2+ escargots vers la même position
                  if snailPosition in liste:
                      removeIndex.append(combIndex)
                      break  

                  #Flag d'un mouvement vers du venin      
                  elif queue[combIndex][snailIndex][1] in state.venoms :
                      removeIndex.append(combIndex) 
                      break 

                  #Flag d'un mouvement vers la position actuelle d'un escargot   
                  elif queue[combIndex][snailIndex][1] in otherSnailsPositions :
                      removeIndex.append(combIndex)
                      break  
          

        # Rejet des combinaisons flagées
        for index in removeIndex:
            queue[index] = None
        while None in queue:
            queue.remove(None) 
                
        ## Créations des états possibles    
        for combIndex in range(len(queue)):
            newMovements = []
            for snailIndex in range(len(queue[combIndex])):
                newMovements.append(queue[combIndex][snailIndex][0])

            new_states.append(state.move(newMovements))

        return new_states


    """
    Résolution par recherche en largeur
    """
    def solve(self, state):
        to_visit = set()
        fifo = deque([state])
        to_visit.add(state)
        # DONE
        nb_statesVisited = 0
        while fifo:
            parentState = fifo.popleft()
            nb_statesVisited += 1
            sucess = self.success(parentState)
            if sucess:
                print("Nombre d'états visités: {}".format(nb_statesVisited))
                return parentState

            childrenStates = self.possible_moves(parentState)
            for child in childrenStates:
                if (child not in to_visit):
                    fifo.append(child)   
                    to_visit.add(child)

                    
    """
    Estimation du nombre de coups restants. 
    """
    def estimee1(self, state):
        # DONE
        h = []
        for pos, exit in zip(state.pos, self.exits):
          h.append(pos.distance(exit))

        return max(h)

    """
    Estimation du nombre de coups restants (heuristique améliorée). 
    """
    
    def estimee2(self, state):
        # DONE
        h =[]
        for i in range(len(state.pos)):
            if state.d[i] == 0 and state.pos[i] != self.exits[i]:
               # Si l'escargot reste sur place sans avoir atteint sa sortie, on ajoute 1 à l'estimé
                h.append(state.pos[i].distance(self.exits[i])+1) 
            else:
                h.append(state.pos[i].distance(self.exits[i]))   

        return max(h)


    """
    Algorithme A*
    """          
    def solve_Astar(self, state):
        to_visit = set()
        to_visit.add(state)
        
        priority_queue = []
        state.h = self.estimee1(state)
        heapq.heappush(priority_queue, state)

        # DONE
        success = False
        nb_statesVisited = 0
        while not success :
            if (priority_queue == []):
                print("priority_queue is empty")
                return None
            parentState = heapq.heappop(priority_queue)
            nb_statesVisited += 1
            success = self.success(parentState)
            if success:
                print("Nombre d'état visités: {}".format(nb_statesVisited))
                return parentState

            childenStates = self.possible_moves(parentState)
            for child in childenStates:
                if (child not in to_visit):
                    child.h = self.estimee1(child)   
                    heapq.heappush(priority_queue, child)
                    to_visit.add(child)

        return None  

    """
    Algorithme A* avec heuristique améliorée
    """  
    def solve_Astar2(self, state):
        to_visit = set()
        to_visit.add(state)
        
        priority_queue = []
        #state.h = self.estimee2(state)
        heapq.heappush(priority_queue, state)

        # DONE
        success = False
        nb_statesVisited = 0  
        while not success :
            if (priority_queue == []):
                print("priority_queue is empty")
                return None

            parentState = heapq.heappop(priority_queue)
            nb_statesVisited += 1
            sucess = self.success(parentState)
            if sucess:
                print("Nombre d'état visités: {}".format(nb_statesVisited))
                return parentState

            to_visit.add(parentState)
            childenStates = self.possible_moves(parentState)
            for child in childenStates:
                if (child not in to_visit):
                    child.h = self.estimee2(child)
                    heapq.heappush(priority_queue, child)
                    to_visit.add(child)

        return None  
       


    """
    Extra function that generates the string of a single step of the solution
    """                 
    def get_print_solution_state(self, state):
        direction = [" vers le centre"," vers le haut"," vers la droite"," vers le bas"," vers la gauche"]
        msg = ""
        msg += str(state.nb_moves)
        msg += ". Escargot "
        for i in range(len(state.d)):
            msg += chr(i + 65) #to get the ascii code of A, B, C, etc.
            msg += direction[state.d[i]]
            if state.pos[i] == self.exits[i]:  #MAH Si l'escargot est à sa sortie
              msg += " (exit)"
            if i == len(state.d)-1:
                msg += ". "
            else:
                msg += ", Escargot "
        return msg


    """
    Imprime les déplacements des escargots à chaque changement d'état pour atteindre
    l'état final
    """  
    def print_solution(self, state):
        # DONE
        solution = []
        printingState = state
        solutionSize = printingState.nb_moves
        for i in range(solutionSize): #every step is added to the whole solution
            solution.append(self.get_print_solution_state(printingState))
            printingState = printingState.prev

        for i in range(len(solution)):   
            print(solution[solutionSize-i-1]) #solution steps are printed in reverse order (to be displayed correctly)
        return 0

    def print_labyrinth(self, state, show_all=True):

        nb_rows = self.nb_lines
        nb_cols = self.nb_columns
        snails_str = ["L_a", "L_b", "L_c", "L_d", "L_e"]
        venom_str = "x"
        exits_str = ["[E_a]","[E_b]","[E_c]","[E_d]","[E_e]"]
        colors = ["red","blue","green","pink","yelllow"]

        if state.prev == None:
            # Prepare table
            cell_text = [["" for j in range(nb_cols+1)] for i in range(nb_rows)]

            fig, ax = plt.subplots()
            fig.set_size_inches((nb_cols+1)*.5,(nb_rows+1)*.5)
            ax.axis('tight')
            ax.axis('off')
            table = ax.table(cellText=cell_text,cellLoc='center', colColours=["gray"]*(nb_cols+1),
                                colWidths=[0.2 for j in range(nb_cols+1)],
                                colLabels=[""]+['$\\bf{%d}$' % val for val in range(nb_cols)],loc='center')
            table.set_fontsize(13)
            table.auto_set_font_size(False)

            cellDict = table.get_celld()
            for i in range(nb_cols+1):
                for j in range(nb_rows+1):
                    cellDict[(j,i)].set_height(.2)

            for i in range(nb_rows+1):
                table[(i, 0)].visible_edges = "open"
            for i in range(nb_cols+1):
                table[(0, i)].visible_edges = "open"
            for i in range(nb_rows):
                table[(i+1, 0)].get_text().set_text("$\\bf{%d}$" % i)

            # Walls
            for i in self.walls:
                table[(i.x+1,i.y+1)].set_facecolor("black")
            # Exits
            for index,i in enumerate(self.exits):
                table[(i.x+1, i.y+1)].get_text().set_text(exits_str[index])
                table[(i.x+1, i.y+1)].get_text().set_color(colors[index])
            # Snails
            for index,snail in enumerate(state.pos):
                table[(snail.x+1, snail.y+1)].get_text().set_text(snails_str[index])
                table[(snail.x+1, snail.y+1)].get_text().set_color(colors[index])
            
            if not show_all:
                clear_output(wait=True)
            fig.suptitle('Snail labyrinth (step = %d)' % 0, x=0.5, y=1.2,fontsize=16)
            display(fig)
            if not show_all:
                time.sleep(1)

            return fig,table,1

        fig,table, n = self.print_labyrinth(state.prev, show_all)

        # Snails
        for index,snail in enumerate(state.pos):
            table[(state.prev.pos[index].x+1, state.prev.pos[index].y+1)].get_text().set_text(venom_str) # Venom
            table[(snail.x+1, snail.y+1)].get_text().set_text(snails_str[index])
            table[(snail.x+1, snail.y+1)].get_text().set_color(colors[index])

        if not show_all:
            clear_output(wait=True)
        fig.suptitle('Snail labyrinth (step = %d)' % n, x=0.5, y=1.2, fontsize=16)
        display(fig)
        if not show_all:
            time.sleep(1)

        return fig,table, n+1


   #### Implémentation
   - Complétez la méthode ***success*** de la classe **Labyrinth** qui indique si un état est final. Cette méthode vérifie si toutes les escargots sont à leur position de sortie. Vous devriez obtenir les résultats suivants:
```
True
True
False
```

In [4]:
def test2():
    b = True
    lb = Labyrinth(5, 7,
                  [Position(0,4), Position(0,6)],
                  [Position(0,3), Position(1,3), Position(2,3), Position(2,2), Position(2,4)])
    s = State([Position(0,0), Position(3,0)])
    s = s.move([3,3]).move([3,2]).move([2,2]).move([3,2]).move([2,2]).move([2,2]).move([2,2]).move([2,1]).move([1,1]).move([1,1]).move([1,1]).move([4,0])
    b = b and lb.success(s)
    print(lb.success(s))

    lb = Labyrinth(8,8,
                   [Position(0,4), Position(0,6), Position(7,0)],
                   [Position(0,3), Position(1,3), Position(2,3), Position(2,2), Position(2,4), Position(6,2), Position(6,4), Position(6,5), Position(6,6), Position(7,1), Position(7,2), Position(7,3), Position(7,4)])
    s = State([Position(0,0), Position(3,0), Position(7,7)])
    s = s.move([3,3,1]).move([3,2,1]).move([2,2,4]).move([3,2,4]).move([2,2,4]).move([2,2,4]).move([2,2,4]).move([2,1,4]).move([1,1,4]).move([1,1,3]).move([1,1,3]).move([4,0,0])
    b = b and lb.success(s)
    print(lb.success(s))
    s = s.move([2,2,2])
    b = b and not lb.success(s)
    print(lb.success(s))
    print("\nrésultat correct" if b else "\nmauvais résultat")

test2()

True
True
False

résultat correct


#### Implémentation 
   - Nous voulons utiliser une matrice $n \times m$ `free_pos` indiquant les cases du labyrinthe qui sont libres. Complétez la méthode **init_positions(State)** qui initialise la matrice `free_pos` en fonction de l'état donné.
   Par convention, *True* indique que la case est libre, et  `free_pos[i,j]` représente la case (ligne,colonne) du labyrinthe.

Exécutez la cellule suivante pour tester la fonction ***init_positions(s)*** de votre code. Vous devriez obtenir les résultats suivants:
```
[[False  True  True False  True  True  True]
 [ True  True  True False  True  True  True]
 [ True  True False False False  True  True]
 [False  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]]

 [[False False False False  True  True  True]
 [False False False False  True  True  True]
 [False False False False False  True  True]
 [False False  True  True  True  True  True]
 [ True  True  True  True  True  True  True]]
```

In [5]:
def test3():
    lb = Labyrinth(5, 7,
                  [Position(0,4), Position(0,6)],
                  [Position(0,3), Position(1,3), Position(2,3), Position(2,2), Position(2,4)])
    s = State([Position(0,0), Position(3,0)])
    lb.init_positions(s)
    b = True
    print(lb.free_pos)
    ans = [[False,True,True,False,True,True,True],[True,True,True,False,True,True,True],
          [True,True,False,False,False,True,True],[False,True,True,True,True,True,True],
          [True,True,True,True,True,True,True]]
    result = lb.free_pos == ans
    for i in range(5):
        for j in range(7):
            b = b and result[i,j]
    print("\n", "résultat correct" if b else "mauvais résultat", "\n")
    s = s.move([2,2]).move([2,1]).move([3,4]).move([4,0]).move([4,0])
    lb.init_positions(s)
    b = True
    print(lb.free_pos)
    ans = [[False,False,False,False,True,True,True],[False,False,False,False,True,True,True],
          [False,False,False,False,False,True,True],[False,False,True,True,True,True,True],
          [True,True,True,True,True,True,True]]
    result = lb.free_pos == ans
    for i in range(5):
        for j in range(7):
            b = b and result[i,j]
    print("\n", "résultat correct" if b else "mauvais résultat", "\n")
            
test3()

[[False  True  True False  True  True  True]
 [ True  True  True False  True  True  True]
 [ True  True False False False  True  True]
 [False  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]]

 résultat correct 

[[False False False False  True  True  True]
 [False False False False  True  True  True]
 [False False False False False  True  True]
 [False False  True  True  True  True  True]
 [ True  True  True  True  True  True  True]]

 résultat correct 



#### Implémentation 
   - Complétez la méthode **possible_moves_snail(snail_position, exit_position, other_exits)** qui à partir de la position d'un escargot, renvoie l'ensemble des mouvements possibles ainsi que leur destination. Cet ensemble est représenté par une liste de `tuple(mouvement,position)`; l'ordre dans la liste n'a pas d'importance. 
   - Complétez la méthode **possible_moves(state)**  qui combine les possibilités de déplacement de toutes les escargots, donné par la fonction **possible_moves_snail**, et renvoie l'ensemble d'états qui peuvent être atteints à partir de l'état *state*. Cet ensemble est représenté par une liste d'états (*states*); l'ordre dans la liste n'est pas important. 
   
**Attention:** comment indiqué précédemment, deux escargots ne peuvent pas se déplacer au même endroit. De plus, on considère qu'il y a un mouvement lorsqu'au moins un escargot se déplace. Une situation où aucune escargot ne se déplace n'est ainsi pas considérée comme un mouvement. Finalement,
notez qu'une escargot ne peut pas se déplacer à une sortie d'un autre escargot.

Exécutez la cellule suivante pour tester votre code. Elle testera les deux fonctions ***possible_moves_snail(snail_position, exit_position, other_exits)*** et ***possible_moves(state)***. Vous devriez obtenir les résultats suivants:
```
[(1, (0,1)), (4, (1,0)), (0, (1,1))]
[(2, (2,2)), (3, (3,1)), (4, (2,0)), (0, (2,1))]
[(1, (0,3)), (0, (1,3))]
11
19
35
```
Il y a ainsi 11 mouvements possibles à partir de l'état représenté par l'image trouvée au début du TP. Il y a 19 mouvements possibles à partir de l'état suivant où les escargots sont se sont déplacées d'une case vers la droite.

In [6]:
def test4():
    b = True
    lb = Labyrinth(4, 4,
                    [Position(2,3), Position(1,2), Position(0,0)],
                    [])
    s0 = State([Position(1,1), Position(2,1), Position(1,3)])
    lb.init_positions(s0)
    possible1 = lb.possible_moves_snail(Position(1,1),Position(2,3),[Position(1,2), Position(0,0)])
    print(possible1)
    ver = True
    for i in possible1:
        if i not in [(1,Position(0,1)), (4,Position(1,0)), (0,Position(1,1))]:
            ver = False
    b = b and len(possible1) == len([(1,Position(0,1)), (4,Position(1,0)), (0,Position(1,1))]) and ver
    possible2 = lb.possible_moves_snail(Position(2,1),Position(1,2),[Position(2,3), Position(0,0)])
    print(possible2)
    ver = True
    for i in possible2:
        if i not in [(2, Position(2,2)), (3, Position(3,1)), (4, Position(2,0)), (0, Position(2,1))]:
            ver = False
    b = b and len(possible2) == len([(2, Position(2,2)), (3, Position(3,1)), (4, Position(2,0)), (0, Position(2,1))]) and ver
    possible3 = lb.possible_moves_snail(Position(1,3),Position(0,0),[Position(2,3), Position(1,2)])
    print(possible3)
    ver = True
    for i in possible3:
        if i not in [(1, Position(0,3)), (0, Position(1,3))]:
            ver = False
    b = b and len(possible3) == len([(1, Position(0,3)), (0, Position(1,3))]) and ver
    lb = Labyrinth(5, 7,
                  [Position(0,4), Position(0,6)],
                  [Position(0,3), Position(1,3), Position(2,3), Position(2,2), Position(2,4)])
    s = State([Position(0,0), Position(3,0)])
    s2 = State([Position(0,1), Position(3,1)])
    s3 = State([Position(3,5), Position(4,4), Position(1,4)])
    l1 = len(lb.possible_moves(s))
    print(l1)
    b = b and l1 == 11
    l2 = len(lb.possible_moves(s2))
    print(l2)
    b = b and l2 == 19
    lb = Labyrinth(5, 7,
                  [Position(0,4), Position(0,6), Position(0,2)],
                  [Position(0,3), Position(1,3), Position(2,3), Position(2,2), Position(2,4)])
    l3 = len(lb.possible_moves(s3))
    print(l3)
    b = b and l3 == 35
    print("\n", "résultat correct" if b else "mauvais résultat", "\n")
test4()

[(1, (0,1)), (4, (1,0)), (0, (1,1))]
[(2, (2,2)), (3, (3,1)), (4, (2,0)), (0, (2,1))]
[(1, (0,3)), (0, (1,3))]
11
19
35

 résultat correct 



## 3. À la recherche d'une solution (30pts)

Il est maintenant possible de chercher une solution avec notre implémentation actuelle. Il y a deux idées principales: 

1. Nous cherchons la solution la plus rapide, il nous faut énumérer tous les états pour les ordonner en ordre croissant en terme de nombre de mouvements. On va donc utiliser une file de type FIFO. 
2. Il est nécessaire de mémoriser les états que l'on a déjà trouvé. Pour cela on va utiliser un ensemble hash. Il s'agit d'utiliser une variable locale **to_visit** de type *set*.

On peut représenter un ensemble d'état par un arbre dont la racine est l'état initial et les nœuds correspondent aux états `s`. La relation entre chaque état est de type `s` *est fils de* `p`, où `s` et `p` sont deux états. Autrement dit, `s` peut être obtenu à partir de `p` en un mouvement.

On cherche à faire une recherche en largeur sur cet arbre. On peut donc représenter la file par une liste (FIFO). La file commence avec l'état initial. Tant que la liste n'est pas vide, on extrait le premier état. Si l'état est final, on termine l'exécution de l'algorithme. Dans le cas contraire, on ajoute ses fils non visités à la fin de la liste. On ajoute ensuite cet état dans la liste `to_visit`.

#### Implementation 
   - Complétez la méthode **solve** en implémentant cet algorithme. La méthode doit renvoyer un état final (las limaces doivent être à leur sortie). 
   - Complétez la méthode **print_solution(state)** qui affiche une solution à partir d'un état final `state`. Il faut afficher les mouvements des escargots dans le bon ordre.

Utilisez la cellule suivante pour tester votre code. Vous devriez obtenir le résultat suivant:  
Il y a 7 mouvements à faire pour résoudre le problème de la figure de départ. **Note** Votre solution peut être différente de celle présentée ici, cela dépend de la méthode de construction de la liste dans la méthode ``possible_moves``. Cependant votre solution doit avoir le même nombre de mouvements (7).

![example7](https://imgur.com/6nXn3mp.png)

```
1. Escargot A vers la gauche, Escargot B vers la droite, Escargot C vers le haut.
2. Escargot A vers le bas, Escargot B vers le haut, Escargot C vers la gauche.
3. Escargot A vers le bas, Escargot B vers le centre, Escargot C vers la gauche.
4. Escargot A vers la droite, Escargot B vers le centre, Escargot C vers la gauche.
5. Escargot A vers la droite, Escargot B vers le centre, Escargot C vers le centre.
6. Escargot A vers la droite, Escargot B vers le centre, Escargot C vers le centre.
7. Escargot A vers le haut, Escargot B vers le centre, Escargot C vers le centre.
```

In [12]:
def solve7():
    lb = Labyrinth(4, 4,
                    [Position(2,3), Position(1,2), Position(0,0)],
                    [])
    s0 = State([Position(1,1), Position(2,1), Position(1,3)])
    s = %time lb.solve(s0)
    lb.print_solution(s)
#     Si vous voulez visualiser les résultats, décommenter la ligne ci-dessous.
#     lb.print_labyrinth(s, show_all=True)

def solve9():
    lb = Labyrinth(5, 5,
                    [Position(3,4), Position(1,3)],
                    [])
    s0 = State([Position(2,0), Position(3,0)])
    s = %time lb.solve(s0)
    lb.print_solution(s)
#     Si vous voulez visualiser les résultats, décommenter la ligne ci-dessous.
#     lb.print_labyrinth(s, show_all=True)

def solve12():
    lb = Labyrinth(5, 7,
                [Position(0,4), Position(0,6)],
                [Position(0,3), Position(1,3), Position(2,3), Position(2,2), Position(2,4)])
    s0 = State([Position(0,0), Position(3,0)])
    s = %time lb.solve(s0)
    lb.print_solution(s)
#     Si vous voulez visualiser les résultats, décommenter la ligne ci-dessous.
#     lb.print_labyrinth(s, show_all=True)

 
solve7()  # EXÉCUTION -> 230 ms
print("\n")
solve9()  # EXÉCUTION -> 41 s
print("\n")
solve12() # EXÉCUTION -> 5 min 


Nombre d'états visités: 533
CPU times: user 237 ms, sys: 1.95 ms, total: 239 ms
Wall time: 240 ms
1. Escargot A vers la gauche, Escargot B vers le centre, Escargot C vers le haut. 
2. Escargot A vers le bas, Escargot B vers le centre, Escargot C vers la gauche. 
3. Escargot A vers le bas, Escargot B vers le centre, Escargot C vers la gauche. 
4. Escargot A vers la droite, Escargot B vers le centre, Escargot C vers la gauche (exit). 
5. Escargot A vers la droite, Escargot B vers le centre, Escargot C vers le centre (exit). 
6. Escargot A vers la droite, Escargot B vers la droite, Escargot C vers le centre (exit). 
7. Escargot A vers le haut (exit), Escargot B vers le haut (exit), Escargot C vers le centre (exit). 


Nombre d'états visités: 23270
CPU times: user 40.8 s, sys: 123 ms, total: 40.9 s
Wall time: 41 s
1. Escargot A vers la droite, Escargot B vers la droite. 
2. Escargot A vers la droite, Escargot B vers la droite. 
3. Escargot A vers le haut, Escargot B vers la droite. 
4. Esc

#### Tests
Voici deux autres problèmes à résoudre : le premier doit être résolu en 9 mouvement; le second en 12.
<table><tr>
<td> <img src="https://i.imgur.com/QCf38qc.png" alt="Drawing"/> </td>
<td> <img src="https://i.imgur.com/3W4iDSk.png" alt="Drawing"/> </td>
</tr></table>

## 4. A* algorithme (35pts)

La méthode proposée précédemment utilise une recherche en largeur pour trouver la solution. Une solution plus rapide peut être mise en place avec un algorithme `A*`. L'idée principale de l'algorithme A* est d'insérer de *l'intelligence artificielle* à la méthode de résolution. Ceci est fait en utilisant une fonction d'estimation qui permet d'évaluer la distance entre un état et la solution. 

Au lieu d'utiliser une approche *first-in first-out (premier entré, premier sorti)* lors de la visite des états, l'algorithme A* défini une valeur de priorité pour chaque état, à partir de la fonction d'estimation, et commence par visiter les états les plus prioritaires. Pour cela, on utilise une structure de données bien plus appropriée : une file de priorité.

En plus de la fonction d'estimation, A* doit aussi tenir compte d'une autre information importante : le coût actuel de l'état. Ce coût est tout simplement, dans notre cas, la distance entre l'état actuel et l'état initial. Ces deux éléments mis ensembles permettent de calculer la fonction d'estimation heuristique ***f*** d'un état `s`, comme la somme du coût actuel de l'état ***c*** et de l'estimation du coût restant ***h***. 

<center> f(s) = coût(s) + h(s) </center>

**Mise en place de l'algorithme**

- Il nous faut donc calculer le nombre de mouvements nécessaires pour atteindre l'état actuel, `coût(s)`. Pour le calculer, créez une variable `nb_moves` dans la classe `State` qui enregistre le nombre de mouvements effectués pour arriver à cet état. Cette variable commence à 0 et doit être incrémentée à chaque fois que l'on crée un nouvel état en utilisant la fonction `move(d)`.

Le coût de la fonction d'estimation est représenté par la variable `h`. Cette fonction estime le nombre de coups restants. Cette composante joue un rôle important dans l'efficacité de l'algorithme A*. Cette variable est utilisée dans la fonction `__lt__` de la classe *State*, qui compare la fonction d'estimation ` f` de deux états. 

Il est donc important de créer une bonne fonction pour résoudre le problème. Plus cette fonction est efficace, moins d'états inutiles seront visités. Cette fonction doit par contre toujours renvoyer un minorant du nombre de coups restant. On va analyser deux fonctions différentes dans ce TP.

#### Fonction 1: à partir de la distance entre les escargots et leur sortie.
- Complétez dans la classe State, la fonction ``estimee1()`` qui renvoie la distance plus grande entre les escargots et leur sortie.
- Maintenant complétez la méthode `solve_Astar(state)` pour résoudre le problème de la même façon qu'au début du TP, mais en utilisant la file de priorité à la place d'une file ordinaire.
- Modifiez les fonctions `solve_Astar(state)` et `solve(state)` de façon à compter et afficher le nombre d'états visités lors de la résolution.
- Modifiez le code des fonctions `solve7(),solve9(),solve12()` pour résoudre les problèmes à l'aide de l'algorithme A*. Vérifiez que les solutions sont obtenues en autant de mouvements qu'avec le premier algorithme, mais avec un nombre inférieur d'états visités. 

N.B: Vous pouvez ajouter des cellules pour faire le test ou afficher les resultats des algorithmes dans une même cellule. Veillez à bien identifier les résultats affichés.




In [10]:
def solve7():
    lb = Labyrinth(4, 4,
                    [Position(2,3), Position(1,2), Position(0,0)],
                    [])


    s0 = State([Position(1,1), Position(2,1), Position(1,3)])
    s = %time lb.solve_Astar(s0)
    lb.print_solution(s)
#     Si vous voulez visualiser les résultats, décommenter la ligne ci-dessous.
#    lb.print_labyrinth(s, show_all=True)

def solve9():
    lb = Labyrinth(5, 5,
                    [Position(3,4), Position(1,3)],
                    [])
    s0 = State([Position(2,0), Position(3,0)])
    s = %time lb.solve_Astar(s0)
    lb.print_solution(s)
#     Si vous voulez visualiser les résultats, décommenter la ligne ci-dessous.
#     lb.print_labyrinth(s, show_all=True)

def solve12():
    lb = Labyrinth(5, 7,
                [Position(0,4), Position(0,6)],
                [Position(0,3), Position(1,3), Position(2,3), Position(2,2), Position(2,4)])

    s0 = State([Position(0,0), Position(3,0)])
    s = %time lb.solve_Astar(s0)
    lb.print_solution(s)
#     Si vous voulez visualiser les résultats, décommenter la ligne ci-dessous.
#     lb.print_labyrinth(s, show_all=True)

 
solve7()  # EXÉCUTION -> 160 ms
print("\n")
solve9()  # EXÉCUTION -> 600 ms
print("\n")
solve12() # EXÉCUTION -> 6.9 s


Nombre d'état visités: 293
CPU times: user 157 ms, sys: 1 ms, total: 158 ms
Wall time: 161 ms
1. Escargot A vers la gauche, Escargot B vers le centre, Escargot C vers le haut. 
2. Escargot A vers le bas, Escargot B vers le centre, Escargot C vers la gauche. 
3. Escargot A vers le bas, Escargot B vers la droite, Escargot C vers la gauche. 
4. Escargot A vers la droite, Escargot B vers le haut (exit), Escargot C vers la gauche (exit). 
5. Escargot A vers la droite, Escargot B vers le centre (exit), Escargot C vers le centre (exit). 
6. Escargot A vers la droite, Escargot B vers le centre (exit), Escargot C vers le centre (exit). 
7. Escargot A vers le haut (exit), Escargot B vers le centre (exit), Escargot C vers le centre (exit). 


Nombre d'état visités: 799
CPU times: user 576 ms, sys: 3.06 ms, total: 579 ms
Wall time: 581 ms
1. Escargot A vers la droite, Escargot B vers la droite. 
2. Escargot A vers le haut, Escargot B vers la droite. 
3. Escargot A vers le haut, Escargot B vers la 

## 5. Amélioration de l'heuristique (Bonus - 10pts)
Proposez une autre fonction heuristique admissible meilleure que `estimee1` pour le problème et testez-la avec les exemples fournis (`solve7()`, `solve9()` et `solve12()`). Ajoutez une fonction nommée `estimee2` et implémentez votre heuristique. **N'oubliez pas de justifier votre choix d'heuristique.**

**Justification :**
On remarque que dans la majorité des cas il n'est pas nécessaire pour un escargot de rester sur place avant d'avoir atteint sa sortie (mouvement nul). Intuitivement, si un escargot reste sur place, on devra nécessairement effectuer au moins un coup supplémentaire. Malgré tout, on voit dans les solutions précédentes que certains escargots restent sur place. Ainsi, on pourrait essayer de donner une plus faible priorité à ce type de mouvement dans l'arbre de recherche.

Si on éliminait complètement la possibilité pour un escargot de rester sur place, on risquerait de surestimer le nombre de coups dans certains labyrinthes. De plus, si un labyrinthe différent requérait obligatoirement des mouvements sur place, aucune solution ne serait trouvée.

De ce fait, la seconde heuristique s'inspire de la première, mais avec la différence que si un escargot reste sur place (mouvement nul) sans avoir atteint sa sortie, on pénalise l'état en augmentant l'estimation du nombre de coups restants de 1 afin de considérer les autres déplacements d'abord. Ainsi, la priorité sera donnée aux mouvements continus, le nombre de coups ne sera pas surestimé et une topologie nécessitant des arrêts mènera à des solutions.

Tel que démontré à la cellule suivante, le nombre d'états visités est bel et bien réduit par rapport à la première heuristique et le nombre de coups est le même.


In [11]:
def solve7():
    lb = Labyrinth(4, 4,
                    [Position(2,3), Position(1,2), Position(0,0)],
                    [])
    s0 = State([Position(1,1), Position(2,1), Position(1,3)])
    s = %time lb.solve_Astar2(s0)
    lb.print_solution(s)
#     Si vous voulez visualiser les résultats, décommenter la ligne ci-dessous.
#     lb.print_labyrinth(s, show_all=True)

def solve9():
    lb = Labyrinth(5, 5,
                    [Position(3,4), Position(1,3)],
                    [])
    s0 = State([Position(2,0), Position(3,0)])
    s = %time lb.solve_Astar2(s0)
    lb.print_solution(s)
#     Si vous voulez visualiser les résultats, décommenter la ligne ci-dessous.
#     lb.print_labyrinth(s, show_all=True)

def solve12():
    lb = Labyrinth(5, 7,
                [Position(0,4), Position(0,6)],
                [Position(0,3), Position(1,3), Position(2,3), Position(2,2), Position(2,4)])
    s0 = State([Position(0,0), Position(3,0)])
    s = %time lb.solve_Astar2(s0)
    lb.print_solution(s)
#     Si vous voulez visualiser les résultats, décommenter la ligne ci-dessous.
#     lb.print_labyrinth(s, show_all=True)


solve7()  # EXÉCUTION -> 120 ms
print("\n")
solve9()  # EXÉCUTION -> 280 ms
print("\n")
solve12() # EXÉCUTION -> 2.9 s


Nombre d'état visités: 146
CPU times: user 92.8 ms, sys: 0 ns, total: 92.8 ms
Wall time: 92.9 ms
1. Escargot A vers la gauche, Escargot B vers le centre, Escargot C vers le haut. 
2. Escargot A vers le bas, Escargot B vers le centre, Escargot C vers la gauche. 
3. Escargot A vers le bas, Escargot B vers le centre, Escargot C vers le centre. 
4. Escargot A vers la droite, Escargot B vers la droite, Escargot C vers la gauche. 
5. Escargot A vers la droite, Escargot B vers le haut (exit), Escargot C vers la gauche (exit). 
6. Escargot A vers la droite, Escargot B vers le centre (exit), Escargot C vers le centre (exit). 
7. Escargot A vers le haut (exit), Escargot B vers le centre (exit), Escargot C vers le centre (exit). 


Nombre d'état visités: 361
CPU times: user 261 ms, sys: 1.02 ms, total: 262 ms
Wall time: 262 ms
1. Escargot A vers le haut, Escargot B vers la droite. 
2. Escargot A vers le haut, Escargot B vers le haut. 
3. Escargot A vers la droite, Escargot B vers le centre. 
4. E

## Barême
Partie 1: 10 points

Partie 2: 25 points

Partie 3: 30 points

Partie 4: 35 points

Partie 5: 10 points (Bonus)

Pour un total de 110 points sur 100 points.

